Day 16 (https://adventofcode.com/2023/day/16)

In [1]:
import numpy as np
from copy import deepcopy

with open('./inputs/day16.txt', 'r') as f:
    tile_map = np.array([list(line.strip()) for line in f.readlines()])

    beam_history_pt1 = []
    beams = [(0, 0, 'E')]

    while len(beams) > 0:
        beam = beams.pop()
        if beam in beam_history_pt1:
            continue

        i, j = beam[0], beam[1]
        beam_history_pt1.append(beam)
        if (beam[2] == 'E' and tile_map[i, j] in ('/', '|')) or \
            (beam[2] == 'W' and tile_map[i, j] in ('\\', '|')) or \
            (beam[2] == 'N' and tile_map[i, j] in ('.', '|')):
            if i > 0:
                beams.append((i-1, j, 'N'))
        if (beam[2] == 'E' and tile_map[i, j] in ('\\', '|')) or \
            (beam[2] == 'W' and tile_map[i, j] in ('/', '|')) or \
            (beam[2] == 'S' and tile_map[i, j] in ('.', '|')):
            if i+1 < tile_map.shape[0]:
                beams.append((i+1, j, 'S'))
        if (beam[2] == 'S' and tile_map[i, j] in ('/', '-')) or \
            (beam[2] == 'N' and tile_map[i, j] in ('\\', '-')) or \
            (beam[2] == 'W' and tile_map[i, j] in ('.', '-')):
            if j > 0:
                beams.append((i, j-1, 'W'))
        if (beam[2] == 'S' and tile_map[i, j] in ('\\', '-')) or \
            (beam[2] == 'N' and tile_map[i, j] in ('/', '-')) or \
            (beam[2] == 'E' and tile_map[i, j] in ('.', '-')):
            if j+1 < tile_map.shape[1]:
                beams.append((i, j+1, 'E'))

    print('Answer to Day 16, Part 1:', len(set(((b[0], b[1]) for b in beam_history_pt1))))

    def get_beam_path_til_split(init_beam):
        beam_path = [init_beam]
        while True:
            beam = beam_path[-1]
            i, j = beam[0], beam[1]
            if (beam[2] == 'E' and tile_map[i, j] == '/') or \
                (beam[2] == 'W' and tile_map[i, j] == '\\') or \
                (beam[2] == 'N' and tile_map[i, j] in ('.', '|')):
                if i > 0:
                    beam_path.append((i-1, j, 'N'))
                    continue
            elif (beam[2] == 'E' and tile_map[i, j] == '\\') or \
                (beam[2] == 'W' and tile_map[i, j] == '/') or \
                (beam[2] == 'S' and tile_map[i, j] in ('.', '|')):
                if i+1 < tile_map.shape[0]:
                    beam_path.append((i+1, j, 'S'))
                    continue
            elif (beam[2] == 'S' and tile_map[i, j] == '/') or \
                (beam[2] == 'N' and tile_map[i, j] == '\\') or \
                (beam[2] == 'W' and tile_map[i, j] in ('.', '-')):
                if j > 0:
                    beam_path.append((i, j-1, 'W'))
                    continue
            elif (beam[2] == 'S' and tile_map[i, j] == '\\') or \
                (beam[2] == 'N' and tile_map[i, j] == '/') or \
                (beam[2] == 'E' and tile_map[i, j] in ('.', '-')):
                if j+1 < tile_map.shape[1]:
                    beam_path.append((i, j+1, 'E'))
                    continue

            return beam_path, (i, j) if (tile_map[i, j]=='-' and beam[2] in ('N', 'S')) or (tile_map[i, j]=='|' and beam[2] in ('E', 'W')) else None
        
    split_dict = {}
    for i, j in np.argwhere(tile_map=='-'):
        east_path, east_split = get_beam_path_til_split((i, j, 'E'))
        west_path, west_split = get_beam_path_til_split((i, j, 'W'))
        split_dict[(i, j)] = {
            'path': east_path + west_path,
            'splits': [s for s in [east_split, west_split] if s is not None]
        }

    for i, j in np.argwhere(tile_map=='|'):
        north_path, north_split = get_beam_path_til_split((i, j, 'N'))
        south_path, south_split = get_beam_path_til_split((i, j, 'S'))
        split_dict[(i, j)] = {
            'path': north_path + south_path,
            'splits': [s for s in [north_split, south_split] if s is not None]
        }

    def get_beam_path(beam):
        path, split = get_beam_path_til_split(beam)
        splits = set([split])
        last_num_splits = 0
        while len(splits) != last_num_splits:
            last_num_splits = len(splits)
            for split in deepcopy(splits):
                if split is None:
                    continue

                splits.update(split_dict[split]['splits'])

        for s in splits:
            if split is None:
                continue
            path += split_dict[s]['path']

        return path
    
    init_beams = [(0, j, 'S') for j in range(tile_map.shape[1])]
    init_beams += [(tile_map.shape[0]-1, j, 'N') for j in range(tile_map.shape[1])]
    init_beams += [(i, 0, 'E') for i in range(tile_map.shape[0])]
    init_beams += [(i, tile_map.shape[1]-1, 'W') for i in range(tile_map.shape[0])]

    num_tiles = [len(set((b[0], b[1]) for b in get_beam_path(init_beam))) for init_beam in init_beams]
    print('Answer to Day 16, Part 2:', max(num_tiles))

Answer to Day 16, Part 1: 8539
Answer to Day 16, Part 2: 8674
